In [1]:
%matplotlib inline
import numpy as np
import matplotlib.tri as Tri
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap
from netCDF4 import Dataset as NetCDFFile 
import netCDF4
import datetime as dt
import pandas as pd
from StringIO import StringIO

In [2]:
import re

def extractURLsatellite(fileURL, satName):
    """
    Function to extract the URLs for a specific satellite from the IMOS URLs list
    
    Parameters:
    -----------
    
    - fileURL : IMOS URLs list as a txt file
    - satName : name of the satellite such as JASON-2 JASON-3 
    
    Ouputs:
    ------
    
    - getFiles : list of URLs for the desired satellite
    
    """
    
    getFiles = []
    
    with open(fileURL) as f:
        for line in f:
            if re.search(r"%s"%satName, line):
                changeURL = re.sub('http://data.aodn.org.au', 'http://thredds.aodn.org.au/thredds/dodsC', line)
                getFiles.append(changeURL)
                
    return getFiles

In [3]:
jason2URL = extractURLsatellite( fileURL = 'IMOSURLs.txt', satName = 'JASON-2')

jason3URL = extractURLsatellite( fileURL = 'IMOSURLs.txt', satName = 'JASON-3')

saralURL = extractURLsatellite( fileURL = 'IMOSURLs.txt', satName = 'SARAL')

sentinel3aURL = extractURLsatellite( fileURL = 'IMOSURLs.txt', satName = 'SENTINEL-3A')

cryosat2URL = extractURLsatellite( fileURL = 'IMOSURLs.txt', satName = 'CRYOSAT-2')

enviURL = extractURLsatellite(fileURL = 'IMOSURLs.txt', satName = 'ENVISAT')

geosatURL = extractURLsatellite(fileURL = 'IMOSURLs.txt', satName = 'GEOSAT')

ersURL = extractURLsatellite(fileURL = 'IMOSURLs.txt', satName = 'ERS-2')

gfoURL = extractURLsatellite(fileURL = 'IMOSURLs.txt', satName = 'GFO')

topURL = extractURLsatellite(fileURL = 'IMOSURLs.txt', satName = 'TOPEX')

In [4]:
allURL = []
allURL.append(jason2URL)
allURL.append(jason3URL)
allURL.append(saralURL)
allURL.append(sentinel3aURL)
allURL.append(cryosat2URL)
allURL.append(enviURL)
allURL.append(geosatURL)
allURL.append(ersURL)
allURL.append(gfoURL)
allURL.append(topURL)

In [ ]:
ncs = NetCDFFile(allURL[0][0])
lats = ncs.variables['LATITUDE'][:]
lons = ncs.variables['LONGITUDE'][:]

In [ ]:
time_var = allURL.variables['TIME']
dtime = netCDF4.num2date(time_var[:],time_var.units)


In [ ]:
ncs = NetCDFFile(jason2URL[0])
ncs.variables['TIME'][0]
dtime2 = netCDF4.num2date(ncs.variables['TIME'][0:150],time_var.units)
print dtime2

In [ ]:
ncs = allURL[0]
dtime2 = netCDF4.num2date(ncs[0:],units = "days since 1985-01-01 00:00:00 UTC", calendar='gregorian')
print dtime2

In [5]:
boxLat = []
boxLon = []
boxWh = []
# boxT = []

In [43]:
#CBG coordinates Box 1
latmin = -24.5 
latmax = -23.0  
lonmin = 152.5 
lonmax = 155.5 

if latmin>latmax:
    print('Error wrong definition of min and max lat!!!')

if lonmin>lonmax:
    print('Error wrong definition of min and max lon!!!')

In [64]:
# Dates to pick
start_date = dt.datetime(2010,1,1)
end_date = dt.datetime(2010,12,1)

In [66]:
for u in range(len(allURL)):
    urlON = allURL[u]
    for k in range(len(urlON)):
        ncs = NetCDFFile(urlON[k])
        lats = ncs.variables['LATITUDE'][:]
        lons = ncs.variables['LONGITUDE'][:]
        if u == 2:
            wh = ncs.variables['SWH_KA_CAL'][:]
        else:
            wh = ncs.variables['SWH_KU_CAL'][:]

        # Get desired time interval  
        time_var = ncs.variables['TIME']
        id_st = netCDF4.date2index(start_date,time_var,select='nearest')
        id_et = netCDF4.date2index(end_date,time_var,select='nearest')
        if id_st<id_et:
            itime = netCDF4.date2index(start_date,time_var,select='after')
            itime2 = netCDF4.date2index(end_date,time_var,select='before')
            
#         dtime = netCDF4.num2date(time_var[itime],time_var.units)
#         daystr = dtime.strftime('%Y-%b-%d %H:%M:%S')
#         print 'record start time:',daystr,itime

#         dtime2 = netCDF4.num2date(time_var[itime2],time_var.units)
#         dayend = dtime2.strftime('%Y-%b-%d %H:%M:%S')
#         print 'record end time:',dayend,itime2

            # Check if satellite points are in the box
            id1 = lats>latmin
            id2 = lats<latmax
            id3 = lons>lonmin
            id4 = lons<lonmax
            id1 = id1*1
            id2 = id2*1
            id3 = id3*1
            id4 = id4*1
            tot = id1+id2+id3+id4
            # If this is the case then the sum should be 4
            inside = np.where(tot==4)[0]
            idtime = np.arange(id_st,id_et+1)

            idin = np.intersect1d(inside, idtime)

            if len(idin)>0:
                boxLat.append(lats[idin])
                boxLon.append(lons[idin])
                boxWh.append(wh[idin])

In [82]:
print len(idin>0)
print len(idin<0)

0
0


In [93]:
dtime2 = netCDF4.num2date(time_var[idin],time_var.units)
dayend = dtime2.strftime('%Y-%b-%d %H:%M:%S')
print 'Jason2 record end time:',dayend

IndexError: list index out of range

In [87]:
end = dt.datetime(2019,2,1)
# Get desired time step  
time_var = ncJ2.variables['TIME']
itime2 = netCDF4.date2index(end,time_var,select='nearest')
dtime2 = netCDF4.num2date(time_var[itime2],time_var.units)
dayend = dtime2.strftime('%Y-%b-%d %H:%M:%S')
#h=nc.variables['SWH_KU_CAL']
print 'Jason2 record end time:',dayend

ValueError: could not convert string to float: idin

In [67]:
len(boxLat)

76

In [68]:
#Did I need this?

# u=0
# urlON = allURL[u]
# k=0
# ncs = NetCDFFile(urlON[k])
# t = ncs.variables['TIME'][:]
# wh = ncs.variables['SWH_KU_CAL'][:]

In [69]:
#wave height pos values
ids = np.where(wh>0)[0]
whpos = wh[ids]
print len(whpos)

7195


In [70]:
print np.mean(whpos)
print max(whpos)
print min (whpos)
print  min(lon)
print max(lon)
print  min(lat)
print max(lat)

1.89246
4.893
0.464
152.501
153.497
-23.5
-23.0003


In [76]:
#Record Start
dtime = netCDF4.num2date(time_var[idin],time_var.units)
daystr = dtime.strftime('%Y-%b-%d %H:%M:%S')
print 'allURL record start time', daystr

#Record end
dtime2 = netCDF4.num2date(time_var[idin],time_var.units)
daystr2 = dtime.strftime('%Y-%b-%d %H:%M:%S')
print 'allURL record end time', daystr2

TypeError: __array__() takes no arguments (1 given)

In [72]:
end = dt.datetime(2019,2,1)
# Get desired time step  
time_var = ncJ2.variables['TIME']
itime2 = netCDF4.date2index(end,time_var,select='nearest')
dtime2 = netCDF4.num2date(time_var[itime2],time_var.units)
dayend = dtime2.strftime('%Y-%b-%d %H:%M:%S')
#h=nc.variables['SWH_KU_CAL']
print 'Jason2 record end time:',dayend

NameError: name 'ncJ2' is not defined